In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, ConfusionMatrixDisplay


In [ ]:
banco_dataset = pd.read_csv('bank.csv')


In [16]:
X = banco_dataset.drop(columns=['target'])
y = banco_dataset['target']
TEST_SIZE = 0.2

X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, random_state=4, test_size=TEST_SIZE)

KeyError: "['target'] not found in axis"